# Import libraries and dependencies

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from analysis_helpers import assign_combined_bucket 
from analysis_helpers import analyze_stocks
import pandas as pd
import utils
import numpy as np

# Load data

In [15]:
# Set the path to the file you'd like to load
file_path = "sp500_data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "shannanl/sp500-dataset",
  file_path,
)
df.rename(columns={df.columns[0]: "day"}, inplace=True)
date_lookup = dict(enumerate(df["day"]))
print("First 5 records:", df.head())

/var/folders/4s/_4bl0c291990qqkpvgrj459r0000gn/T/ipykernel_13919/1772646487.py:5: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:           day  MMM_high   MMM_low  MMM_open  MMM_close  MMM_volume  \
0  2000-01-03  48.25000  47.03125  48.03125    47.1875   2173400.0   
1  2000-01-04  47.40625  45.31250  46.43750    45.3125   2713800.0   
2  2000-01-05  48.12500  45.56250  45.56250    46.6250   3699400.0   
3  2000-01-06  51.25000  47.15625  47.15625    50.3750   5975800.0   
4  2000-01-07  51.90625  49.96875  50.56250    51.3750   4101200.0   

   MMM_adj_close   ABT_high    ABT_low   ABT_open  ...  ZION_open  ZION_close  \
0      27.179523  16.160433  15.599306  15.823756  ...   59.03125     55.5000   
1      26.099533  15.599306  15.150405  15.459024  ...   54.62500     52.8125   
2      26.855530  15.402911  15.066236  15.066236  ...   52.75000     52.7500   
3      29.015484  15.823756  15.178461  15.262630  ...   52.75000     53.5000   
4      29.591490  16.272657  15.487080  15.487080  ...   53.75000     53.6250   

   ZION_volume  ZION_adj_close  ZTS_high  ZTS_low  ZTS_open  ZTS_close  \
0

First, lets analyze which stocks we should focus on. For that, we create buckets in regards to high/low liquidity/volatility, to see how model performance correlates to these stock attributes.

In [16]:
bucket_df = analyze_stocks(df)


# Extract only the first 5 tickers for each quadrant

HV_HL = bucket_df[(bucket_df["vol_bucket"] == "high") & 
                  (bucket_df["liq_bucket"] == "high")].index.tolist()[:5]

HV_LL = bucket_df[(bucket_df["vol_bucket"] == "high") & 
                  (bucket_df["liq_bucket"] == "low")].index.tolist()[:5]

LV_HL = bucket_df[(bucket_df["vol_bucket"] == "low") & 
                  (bucket_df["liq_bucket"] == "high")].index.tolist()[:5]

LV_LL = bucket_df[(bucket_df["vol_bucket"] == "low") & 
                  (bucket_df["liq_bucket"] == "low")].index.tolist()[:5]

# Print the lists
print("High Volatility / High Liquidity:", HV_HL)
print("High Volatility / Low Liquidity:", HV_LL)
print("Low Volatility / High Liquidity:", LV_HL)
print("Low Volatility / Low Liquidity:", LV_LL)




High Volatility / High Liquidity: ['KEY', 'VIAC', 'CCL', 'AAPL', 'GILD']
High Volatility / Low Liquidity: ['RL', 'CMG', 'GNRC', 'KSU', 'ODFL']
Low Volatility / High Liquidity: ['JNJ', 'PEP', 'PG', 'KO', 'NEE']
Low Volatility / Low Liquidity: ['WEC', 'AEE', 'DTE', 'VRSK', 'MKC']


/Users/vinconig/Desktop/PUCP/IAA/Repos/Stonks/analysis_helpers.py:26: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df[close_col].pct_change()


Load all available stock history of a S&P 500 company of your choosing into a separate dataframe. Note: we transform the date to a timestep, because we dont want to train the model on the actual physical point in time. we just need to give it sequential context.

In [17]:
df_apple = utils.load_comp_data(df, "AAPL")
print("Apple stock data:", df_apple.head())

Apple stock data:    timestep  AAPL_high  AAPL_low  AAPL_open  AAPL_close  AAPL_volume  \
0         0   1.004464  0.907924   0.936384    0.999442  535796800.0   
1         1   0.987723  0.903460   0.966518    0.915179  512377600.0   
2         2   0.987165  0.919643   0.926339    0.928571  778321600.0   
3         3   0.955357  0.848214   0.947545    0.848214  767972800.0   
4         4   0.901786  0.852679   0.861607    0.888393  460734400.0   

   AAPL_adj_close  
0        0.858137  
1        0.785788  
2        0.797286  
3        0.728291  
4        0.762789  


/Users/vinconig/Desktop/PUCP/IAA/Repos/Stonks/utils.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result.fillna(method='ffill', inplace=True)


However, of course, we can always get the actual day back, if we need to plot or else, by using the 'timestep' value. This will also work after data was transformed.

In [18]:
date_lookup[42]

'2000-03-03'

Apply feature engineering techniques to stock. As many features use a rolling window, you can decide if you want to cut the first rows, where those values are not calculated using data from the entirety of the window.

In [19]:
window_size = 20
df_apple_fe = utils.compute_features(df_apple, w=window_size, cut=False)
print("Apple stock data with feature engineering:", df_apple_fe.tail())
date_lookup[df_apple_fe.iloc[0]["timestep"]]


Apple stock data with feature engineering:       timestep  AAPL_high  AAPL_low  AAPL_open  AAPL_close  AAPL_volume  \
5456      5477  -0.593097 -0.194214  -0.233947   -0.538862    -1.263644   
5457      5478  -0.289042 -0.358134  -0.702262   -0.493662    -0.899623   
5458      5479  -0.848506 -0.558214  -0.297249   -0.877397    -0.427938   
5459      5480  -1.553588 -1.145604  -0.967597   -1.037309    -0.138639   
5460      5481  -0.323117 -0.158679  -0.546650    0.274642    -0.575012   

      log_return  true_range    rsd_20    ntv_20  ...  bollinger_mid  \
5456   -0.580691   -1.391396 -2.480652 -1.217181  ...      -1.966097   
5457    0.501146    0.031912 -2.115457 -0.829901  ...      -1.827024   
5458   -1.076568   -0.714177 -1.948051 -0.316907  ...      -1.714997   
5459   -0.084412   -0.561067 -1.751735 -0.017193  ...      -1.649681   
5460    1.205837    0.086429 -1.299038 -0.429253  ...      -1.551939   

      bollinger_upper  bollinger_lower  bollinger_width  bollinger_percen

'2000-02-02'